In [ ]:
import cv2
import numpy as np
import os
import random
import matplotlib.pyplot as plt

from sklearn import tree
from time import time

from skimage.transform import integral_image
from skimage.feature import haar_like_feature   
from sklearn import metrics

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_face = os.listdir('/content/drive/MyDrive/faces/face_train/face')
train_face_count = len(train_face)
train_non_face = os.listdir('/content/drive/MyDrive/faces/face_train/non-face') 
train_non_face_count = len(train_face)
test_face = os.listdir('/content/drive/MyDrive/faces/face_test/face')
test_face_count = len(test_face)
test_non_face = os.listdir('/content/drive/MyDrive/faces/face_test/non-face')
test_non_face_count = len(test_non_face)


Further on the work that supposed to be done with validation set is done with the given test set as there was no way of splitting pgm files into validation and training sets.

In [ ]:
DIR="/content/drive/MyDrive/faces/face_train"
CAT=["face","non-face"]
DIR_TEST = "/content/drive/MyDrive/faces/face_test"
CAT_TEST = ["face", "non-face"]
training_data=[]
feature_types = ['type-2-x', 'type-2-y', 'type-3-x', 'type-4']
def init_training_data():
    for cat in CAT:
        path = os.path.join(DIR, cat)
        class_num=CAT.index(cat)
        img_path = os.listdir(path)
        for img in img_path:
            features = []
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img = np.array(img_array)
            img_i = integral_image(img)

            features = haar_like_feature(img_i, 0, 0, img_i.shape[1], img_i.shape[0], feature_types)    
            training_data.append(features)
t_start = time()
init_training_data()
total_time = time() - t_start
print("Time to train features: ", total_time)

Time to train features:  988.5846538543701


In [ ]:
testing_data = []
def init_testing_data():
    for cat in CAT_TEST:
        path = os.path.join(DIR_TEST, cat)
        class_num_test = CAT_TEST.index(cat)
        img_path = os.listdir(path)
        for img in img_path:
            features = []
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img = np.array(img_array)

            img_i = integral_image(img)
            try:
                features = (haar_like_feature(img_i, 0, 0, img_i.shape[1], img_i.shape[0], feature_types))
            except IndexError:
                continue      
            testing_data.append(features)
init_testing_data() 

In [ ]:
y_train = np.array([1] * train_face_count + [0] * train_non_face_count)

In [ ]:
y_test = np.array([1] * test_face_count + [0] * test_non_face_count) 

In [ ]:
abc = AdaBoostClassifier(n_estimators=10, algorithm="SAMME")
t_start = time()
model = abc.fit(training_data, y_train)
time_adaboost = time() - t_start
print("Time to train (AdaBoost):", time_adaboost)

y_pred = model.predict(testing_data)
print("Accuracy (AdaBoost): ", metrics.accuracy_score(y_test, y_pred))

In [ ]:
print(abc.estimators_)
for i in range(len(abc.estimators_)):
    print(f"Tree {i}: ")
    print(f"Weight: {abc.estimator_weights_[i]}")
    print(tree.export_text(abc.estimators_[i]))

In [ ]:
rf = RandomForestClassifier(n_estimators=10, max_depth=None, max_features=100, n_jobs=-1, random_state=0)
t_start = time()
rf.fit(training_data, y_train)
print("Time to train (RandomForests): ")
time_rf = time() - t_start
y_pred_rf = rf.predict(testing_data)
print("Accuracy (Random Forests):", metrics.accuracy_score(y_test, y_pred_rf))

Clearly, according to the training times, the Adaboost algorithm is more efficient that the Random Forest classifier for the Viola and Jones face recognition algorithm.